<a href="https://colab.research.google.com/github/Hirunikathaluwage/RAG/blob/main/Simple_RAG_Application_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.2 MB/s eta 0:

In [4]:
#import necessary libraries
import os
from google.colab import userdata

#initialize language model

In [5]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#initialize the ChatOpenAI model
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

#initialize embedding model

In [6]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
!pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


#Create embedded documents


In [8]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content=" New app nexora 2025 has been launched.",
        METADATA ={"source" "mobile app"},),

    Document(
        page_content=" New desktop app new 2025 has been launched.",
        METADATA ={"source" "desktop app"},),
    Document(
        page_content=" The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals",
        METADATA ={"source" "ai revolution news"},),
]

In [9]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents, embedding=embedding_model
)

#Perform similarity search

In [10]:
results = vectorstore.similarity_search("mobile app")

for results in results:
  print(results.page_content)
  print(results.metadata)



 New app nexora 2025 has been launched.
{}
 New desktop app new 2025 has been launched.
{}
 The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals
{}


In [11]:
results = vectorstore.similarity_search("desktop app")

for results in results:
  print(results.page_content)
  print(results.metadata)

print("\n")

results = vectorstore.similarity_search("ai revolution")
for results in results:
  print(results.page_content)
  print(results.metadata)

 New desktop app new 2025 has been launched.
{}
 New app nexora 2025 has been launched.
{}
 The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals
{}


 The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals
{}
 New app nexora 2025 has been launched.
{}
 New desktop app new 2025 has been launched.
{}


#Embedded query and perform similerity search by vector

In [12]:
query_embedding = embedding_model.embed_query("ai revolution")
query_embedding[:10]

[0.014172401279211044,
 -0.01473250798881054,
 0.040395587682724,
 0.0255782138556242,
 -0.004279301501810551,
 -0.017397258430719376,
 -0.04586087167263031,
 0.037374403327703476,
 -0.02386394701898098,
 0.006335148122161627]

In [13]:
len(query_embedding)  #print the length of the query embedding

1536

Create Retriver


In [14]:
#create a retriever from the vector
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1,},
)

#perform batch retrieva using the retriever
batch_results = retriever.batch(["Matching Learning"])
for result in batch_results:
  for doc in result:
    print(doc.page_content)
    print(doc.metadata)


 The AI revolution continous to transform industries and reshape the global economy.Significant advancments in articals
{}


In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

#define a message template for the chatbot
message = """
answer this question using the provided context only.

{question}

Context:
{context}
"""

#Create a chat prompt template from the message
prompt = ChatPromptTemplate.from_messages([("human", message)])

#Chain retriever and prompt template with LLM

In [19]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [20]:
response = chain.invoke("current mobile apps")
print(response.content)

There is a new desktop app called new 2025 that has been launched.
